# Ball Landing & Pocket Extraction (Inference Notebook)

This notebook:

1. Loads wheel geometry from `wheel_geometry.json`
2. Runs a YOLO model on a roulette video
3. Detects when the **ball** is inside the **halo band**
4. If the ball stays in the halo for **> 3 seconds**, marks **ball landed**
5. Computes the **landing pocket value** (European wheel order by default)

> Adjust paths in the **Config** cell below.


In [ ]:
import json
import math
import numpy
import cv2
import sys
import matplotlib.pyplot as plt
from ultralytics import YOLO
from pathlib import Path

sys.path.append(str(Path.cwd().parent))
from utils.file_dialog_utils import pick_video_cv2, pick_file

# Paths (edit these)
_, VIDEO_PATH = pick_video_cv2() 
GEOMETRY_PATH = r"C:\Users\Gabriel\Documents\Dissertation\Code\notebooks\roulette_cv\wheel_geometry.json"
MODEL_PATH = r"C:\Users\Gabriel\Documents\Dissertation\Code\models\yolo\RD2_Model.pt"

CONF_THRES = 0.35
IOU_THRES  = 0.70
DEVICE     = 0  # 0 for GPU, "cpu" for CPU

BALL_CLASS_ID = 0

LANDING_SECONDS_REQUIRED = 2.5

WHEEL_TYPE = "EU"  # "EU" (37 pockets) or "US" (38 pockets)

In [ ]:
# Load geometry from JSON

with open(GEOMETRY_PATH, "r") as f:
    geom = json.load(f)

cx = float(geom["ellipse"]["cx"])
cy = float(geom["ellipse"]["cy"])
rx = float(geom["ellipse"]["rx"])
ry = float(geom["ellipse"]["ry"])
ellipse_rot_deg = float(geom["ellipse"]["rotation_deg"])  # orientation of ellipse in the image
zero_angle_deg  = float(geom["zero_angle_deg"])           # reference direction to 0 pocket (from calibration frame)

outer_inward_pct = float(geom["halo"]["outer_inward_pct"])
inner_inward_pct = float(geom["halo"]["inner_inward_pct"])

# Halo boundaries expressed in normalized-ellipse space
# r_norm = 1.0 is the fitted ellipse. Moving inward shrinks r_norm.
outer_r_norm = 1.0 - outer_inward_pct
inner_r_norm = 1.0 - inner_inward_pct

if inner_r_norm >= outer_r_norm:
    raise ValueError(
        f"Invalid halo params: inner boundary must be smaller radius than outer boundary. "
        f"Got inner_r_norm={inner_r_norm:.3f}, outer_r_norm={outer_r_norm:.3f}"
    )

print("Loaded geometry:")
print(f"  center=({cx:.2f},{cy:.2f}) rx={rx:.2f} ry={ry:.2f} ellipse_rot_deg={ellipse_rot_deg:.2f}")
print(f"  zero_angle_deg={zero_angle_deg:.2f}")
print(f"  halo band r_norm in [{inner_r_norm:.3f}, {outer_r_norm:.3f}]")

In [ ]:
# Wheel pocket order mapping

EUROPEAN_WHEEL_ORDER = [
    0,
    32, 15, 19, 4, 21, 2, 25, 17, 34,
    6, 27, 13, 36, 11, 30, 8, 23, 10,
    5, 24, 16, 33, 1, 20, 14, 31,
    9, 22, 18, 29, 7, 28, 12,
    35, 3, 26
]

AMERICAN_WHEEL_ORDER = [
    0, 28, 9, 26, 30, 11, 7, 20, 32, 17,
    5, 22, 34, 15, 3, 24, 36, 13, 1, "00",
    27, 10, 25, 29, 12, 8, 19, 31, 18,
    6, 21, 33, 16, 4, 23, 35, 14, 2
]

if WHEEL_TYPE.upper() == "EU":
    wheel_order = EUROPEAN_WHEEL_ORDER
    pockets = 37
elif WHEEL_TYPE.upper() == "US":
    wheel_order = AMERICAN_WHEEL_ORDER
    pockets = 38
else:
    raise ValueError("WHEEL_TYPE must be 'EU' or 'US'")

sector_size_deg = 360.0 / pockets

print(f"Wheel type: {WHEEL_TYPE} pockets={pockets} sector_size={sector_size_deg:.4f}°")

In [ ]:
# Helper functions

def _rotate_point(x: float, y: float, cx: float, cy: float, rot_deg: float):
    """Rotate (x,y) around (cx,cy) by -rot_deg to align with ellipse axes."""
    theta = math.radians(rot_deg)
    tx = x - cx
    ty = y - cy
    cos_t = math.cos(theta)
    sin_t = math.sin(theta)
    # rotation by -theta:
    xr =  tx * cos_t + ty * sin_t
    yr = -tx * sin_t + ty * cos_t
    return xr, yr

def ball_angle_deg(x: float, y: float) -> float:
    """Angle in image coordinates, 0° is +x to the right, increasing CCW (with y inverted)."""
    dx = x - cx
    dy = y - cy
    return (math.degrees(math.atan2(-dy, dx)) % 360.0)

def ball_r_norm(x: float, y: float) -> float:
    """Normalized radius in ellipse-aligned coordinates: r=1 on ellipse, <1 inside."""
    xr, yr = _rotate_point(x, y, cx, cy, ellipse_rot_deg)
    if rx <= 0 or ry <= 0:
        return float("inf")
    return math.sqrt((xr / rx) ** 2 + (yr / ry) ** 2)

def is_in_halo(x: float, y: float) -> bool:
    r = ball_r_norm(x, y)
    return (inner_r_norm <= r <= outer_r_norm)

def pocket_index_from_ball_angle(ball_ang: float) -> int:
    """Compute pocket sector index using zero reference."""
    rel = (ball_ang - zero_angle_deg) % 360.0
    idx = int(rel // sector_size_deg)
    if idx < 0: idx = 0
    if idx >= pockets: idx = pockets - 1
    return idx

def pocket_value_from_index(idx: int):
    return wheel_order[idx]

In [ ]:
# Run model + detect landing + compute pocket

# Open video to get FPS and frame count
cap = cv2.VideoCapture(str(VIDEO_PATH))
if not cap.isOpened():
    raise RuntimeError(f"Could not open video: {VIDEO_PATH}")

fps = cap.get(cv2.CAP_PROP_FPS)
if fps <= 0:
    fps = 30.0  # fallback
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
cap.release()

frames_required = int(round(LANDING_SECONDS_REQUIRED * fps))

print(f"Video FPS={fps:.3f}, frames={frame_count}, landing requires {frames_required} consecutive halo-frames (~{LANDING_SECONDS_REQUIRED}s).")

# Load YOLO model
model = YOLO(str(MODEL_PATH))

# State
consecutive_halo_frames = 0
landed = False
landing_frame_idx = None
landing_ball_xy = None
landing_ball_angle = None
landing_pocket_idx = None
landing_pocket_value = None

frame_idx = -1

# Stream predictions
for r in model.predict(
    source=str(VIDEO_PATH),
    stream=True,
    conf=CONF_THRES,
    iou=IOU_THRES,
    device=DEVICE
):
    frame_idx += 1

    boxes = r.boxes
    if boxes is None or len(boxes) == 0:
        consecutive_halo_frames = 0
        continue

    # Find best ball detection (highest confidence among class BALL_CLASS_ID)
    best = None
    best_conf = -1.0

    cls = boxes.cls.cpu().numpy().astype(int)
    conf = boxes.conf.cpu().numpy()
    xyxy = boxes.xyxy.cpu().numpy()

    for i in range(len(cls)):
        if cls[i] != BALL_CLASS_ID:
            continue
        if conf[i] > best_conf:
            best_conf = float(conf[i])
            best = xyxy[i]

    if best is None:
        consecutive_halo_frames = 0
        continue

    x1, y1, x2, y2 = best
    bx = float((x1 + x2) / 2.0)
    by = float((y1 + y2) / 2.0)

    in_halo = is_in_halo(bx, by)

    if in_halo:
        consecutive_halo_frames += 1
    else:
        consecutive_halo_frames = 0

    # If landed condition met, lock result once
    if (not landed) and (consecutive_halo_frames >= frames_required):
        landed = True
        landing_frame_idx = frame_idx
        landing_ball_xy = (bx, by)
        landing_ball_angle = ball_angle_deg(bx, by)
        landing_pocket_idx = pocket_index_from_ball_angle(landing_ball_angle)
        landing_pocket_value = pocket_value_from_index(landing_pocket_idx)
        break

if not landed:
    print("❌ Landing NOT detected (ball did not stay in halo long enough).")
else:
    landing_time_s = landing_frame_idx / fps
    print("✅ Ball landed!")
    print(f"  landing_frame_idx = {landing_frame_idx}")
    print(f"  landing_time_s    = {landing_time_s:.2f}s")
    print(f"  ball_xy           = ({landing_ball_xy[0]:.1f}, {landing_ball_xy[1]:.1f})")
    print(f"  ball_angle_deg    = {landing_ball_angle:.2f}")
    print(f"  pocket_index      = {landing_pocket_idx}")
    print(f"  pocket_value      = {landing_pocket_value}")


In [ ]:
# Pocket mapping visualization (debug)

def _img_line(overlay, p1, p2, color, thickness=2):
    cv2.line(overlay, (int(p1[0]), int(p1[1])), (int(p2[0]), int(p2[1])), color, thickness)

def _img_text(overlay, text, xy, color=(255,255,255), scale=0.7, thickness=2):
    cv2.putText(
        overlay, str(text), (int(xy[0]), int(xy[1])),
        cv2.FONT_HERSHEY_SIMPLEX, scale, color, thickness, cv2.LINE_AA
    )

def visualize_pocket_mapping(
    frame_index: int = 0,
    radius_norm: float = None,
    show_boundaries: bool = True,
    show_labels: bool = True,
    label_every: int = 1,
    label_scale: float = 0.65
):
    # Reads a frame, draws ellipse + halo, then draws pocket sector boundaries and labels.
    # Labels correspond to wheel_order[k].
    if radius_norm is None:
        radius_norm = (inner_r_norm + outer_r_norm) / 2.0  # middle of the halo band

    cap = cv2.VideoCapture(str(VIDEO_PATH))
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
    ok, frame = cap.read()
    cap.release()
    if not ok:
        raise RuntimeError(f"Failed to read frame {frame_index} from {VIDEO_PATH}")

    overlay = frame.copy()

    # Context: fitted ellipse + halo boundaries
    cv2.ellipse(overlay, (int(cx), int(cy)), (int(rx), int(ry)), ellipse_rot_deg, 0, 360, (0, 255, 0), 2)
    cv2.ellipse(overlay, (int(cx), int(cy)), (int(rx*outer_r_norm), int(ry*outer_r_norm)), ellipse_rot_deg, 0, 360, (255, 255, 0), 2)
    cv2.ellipse(overlay, (int(cx), int(cy)), (int(rx*inner_r_norm), int(ry*inner_r_norm)), ellipse_rot_deg, 0, 360, (0, 255, 255), 2)

    def point_on_scaled_ellipse(angle_deg: float, rnorm: float):
        # Angle is in image coords: 0° to the right, increasing CCW with y inverted.
        ax = math.cos(math.radians(angle_deg))
        ay = -math.sin(math.radians(angle_deg))

        # Build point in ellipse-aligned coords, then rotate into image coords
        x_al = rx * rnorm * ax
        y_al = ry * rnorm * ay

        theta = math.radians(ellipse_rot_deg)
        cos_t = math.cos(theta)
        sin_t = math.sin(theta)

        x_img = cx + (x_al * cos_t - y_al * sin_t)
        y_img = cy + (x_al * sin_t + y_al * cos_t)
        return x_img, y_img

    base = zero_angle_deg  # absolute image angle for the 0-pocket reference

    for k in range(pockets):
        boundary_angle = (base + k * sector_size_deg) % 360.0
        mid_angle = (base + (k + 0.5) * sector_size_deg) % 360.0

        if show_boundaries:
            p_end = point_on_scaled_ellipse(boundary_angle, outer_r_norm * 1.15)
            _img_line(overlay, (cx, cy), p_end, color=(200, 200, 200), thickness=1)

        if show_labels and (k % label_every == 0):
            p_lab = point_on_scaled_ellipse(mid_angle, radius_norm)
            val = wheel_order[k]
            _img_text(overlay, val, p_lab, color=(255, 255, 255), scale=label_scale, thickness=2)

    # Emphasize the zero reference boundary line
    p_zero = point_on_scaled_ellipse(base, outer_r_norm * 1.25)
    _img_line(overlay, (cx, cy), p_zero, color=(0, 255, 255), thickness=3)
    _img_text(overlay, "ZERO_REF", (p_zero[0] + 8, p_zero[1] + 8), color=(0,255,255), scale=0.7, thickness=2)

    # If landing detected, draw ball + its angle ray
    if 'landed' in globals() and landed and landing_ball_xy is not None:
        bx, by = landing_ball_xy
        cv2.circle(overlay, (int(bx), int(by)), 6, (0, 0, 255), -1)
        a = landing_ball_angle
        p_ang = point_on_scaled_ellipse(a, outer_r_norm * 1.25)
        _img_line(overlay, (cx, cy), p_ang, color=(0, 0, 255), thickness=3)
        _img_text(overlay, f"LAND {landing_pocket_value}", (p_ang[0] + 8, p_ang[1] + 8), color=(0,0,255), scale=0.8, thickness=2)

    plt.figure(figsize=(14, 8))
    plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.title("Pocket Mapping Overlay (labels = wheel_order[k])")
    plt.show()

visualize_pocket_mapping(frame_index=0, label_every=1)

In [ ]:
import matplotlib.pyplot as plt

if landed:
    cap = cv2.VideoCapture(str(VIDEO_PATH))
    cap.set(cv2.CAP_PROP_POS_FRAMES, landing_frame_idx)
    ok, frame = cap.read()
    cap.release()

    if ok:
        overlay = frame.copy()

        # Draw fitted ellipse
        cv2.ellipse(overlay, (int(cx), int(cy)), (int(rx), int(ry)), ellipse_rot_deg, 0, 360, (0, 255, 0), 2)

        # Draw halo boundaries (scaled ellipses)
        outer_rx = rx * outer_r_norm
        outer_ry = ry * outer_r_norm
        inner_rx = rx * inner_r_norm
        inner_ry = ry * inner_r_norm

        cv2.ellipse(overlay, (int(cx), int(cy)), (int(outer_rx), int(outer_ry)), ellipse_rot_deg, 0, 360, (255, 255, 0), 2)
        cv2.ellipse(overlay, (int(cx), int(cy)), (int(inner_rx), int(inner_ry)), ellipse_rot_deg, 0, 360, (0, 255, 255), 2)

        # Draw ball point
        bx, by = landing_ball_xy
        cv2.circle(overlay, (int(bx), int(by)), 5, (0, 0, 255), -1)

        text = f"Landed: {landing_pocket_value} (idx={landing_pocket_idx}) t={landing_frame_idx/fps:.2f}s"
        cv2.putText(overlay, text, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2)

        plt.figure(figsize=(12, 7))
        plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title("Landing Frame Debug Overlay")
        plt.show()
    else:
        print("Could not read landing frame for visualization.")
else:
    print("No landing detected; nothing to visualize.")
